In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 42.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 30.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from VAN_new import *
import pandas as pd

def get_column(matrix, i):
    return torch.tensor([matrix[j][i] for j in range(len(matrix))])

def energie1D(spin): 
    spin_copie=spin.clone()
    spin_copie[spin_copie==0]=-1
    spin_copie_1 = torch.roll(spin_copie, -1)
    spin_copie_2 = torch.roll(spin_copie, 1)    
    energie=- torch.sum(spin_copie_1*spin_copie+spin_copie_2*spin_copie)
    return energie

def energie2D(lattice):
    energie = 0 
    for i in range(len(lattice)):
        energie+=energie1D(lattice[0])
    for j in range(len(lattice[0])):
        column = get_column(lattice, j)
        energie+=energie1D(column)
    return energie


def log_prob_energie(beta, energie):
    return -beta*energie


def log_prob_target_energie(spins, beta):
    
    log_probs = torch.ones(spins.shape[0]) * np.log(0.001)
    for i in range(len(log_probs)):
        racine=spins[i].shape[0]
        racine=(int(np.sqrt(racine)))
        lattice = spins[i].reshape(racine, racine)
        log_probs[i] = log_prob_energie(beta, energie2D(lattice))
    return log_probs 



taille=100
betas = np.linspace(0.5, 6.5, 11)
for beta in betas: 
    magnetisations_list=[]
    for i in range(10): 
        print(beta, i)
        mymodel1 = VAN(taille)
        losses = train(mymodel1, lambda x:  log_prob_target_energie(x, beta), batch_size=200, n_iter=1000, lr=0.01)
        mysample=mymodel1.sample(1000)
        magnetisations=[]
        for spin in mysample:
            magnetisations.append(torch.mean(spin))
        plt.hist(magnetisations, bins=20, edgecolor='black') 
        plt.xlabel('Magnetisation du spin')
        plt.ylabel('Nombre de spins')
        plt.title('Magnetisation des spins pour beta =' + str(beta))
        plt.savefig('./figures/magnetisation for beta= '+str(beta)+' test n° ' + str(i) + '.png')
        magnetisations_list.append(magnetisations)
    pd.DataFrame(magnetisations_list).to_csv('.magnetisations/magnetisations for beta= '+str(beta)+'.csv')
        


0.5 0
Epoch 0: 38.393653869628906
Epoch 100: -35.37469482421875
Epoch 200: -7.582620143890381
Epoch 300: -7.943325042724609
Epoch 400: -5.7026753425598145
Epoch 500: -4.3742218017578125
Epoch 600: -2.193530797958374
Epoch 700: -3.9057910442352295
Epoch 800: -4.145415782928467
Epoch 900: 1.1877118349075317
0.5 1
Epoch 0: 34.803585052490234
Epoch 100: -52.017234802246094
Epoch 200: -2.3652377128601074
Epoch 300: -0.16997192800045013
Epoch 400: -10.464598655700684
Epoch 500: -0.7688331007957458
Epoch 600: -4.26547384262085
Epoch 700: -0.06730621308088303
Epoch 800: -0.9461260437965393
Epoch 900: 0.6495862007141113
0.5 2
Epoch 0: 30.5024471282959
Epoch 100: -65.75353240966797
Epoch 200: -20.300500869750977
Epoch 300: -2.4473071098327637
Epoch 400: -2.048814296722412
Epoch 500: -0.6100460886955261
Epoch 600: 1.1543328762054443
Epoch 700: 0.2089821994304657
Epoch 800: -1.896180272102356
Epoch 900: 0.7561715841293335
